In [1]:
import python_vali as vali
import numpy as np
import av

In [2]:
input = "../tests/data/test.mp4"
output = 'output.mp4'

target_w = 640
target_h = 360

In [3]:
# GPU-accelerated decoder
py_dec = vali.PyDecoder(
    input,
    {},
    gpu_id=0)

# GPU-accelerated resizer
py_res = vali.PySurfaceResizer(py_dec.Format, gpu_id=0)

# GPU-accelerated encoder
py_enc = vali.PyNvEncoder(
    {
        "preset": "P4",
        "codec": "h264",
        "s": str(target_w) + 'x' + str(target_h),
        "bitrate": "1M",
        "fps": f"{py_dec.Framerate}",
    },
    gpu_id=0
)

# Muxer
dst_file = av.open(output, 'w')
out_stream = dst_file.add_stream('h264', rate=int(py_dec.Framerate))
out_stream.width = target_w
out_stream.height = target_h

In [4]:
def make_packet(
        video_packet: np.ndarray,
        pkt_data: vali.PacketData,
        out_stream: av.VideoStream) -> av.Packet:
    """Create an AV packet from encoded video data and packet metadata.

    This function takes encoded video data and associated metadata to create a properly
    formatted AV packet that can be written to an output video stream.

    Args:
        video_packet (np.ndarray): A numpy array containing the encoded video packet data
        pkt_data (vali.PacketData): Packet metadata containing presentation and decode timestamps
        out_stream (av.VideoStream): The output video stream to associate with the packet

    Returns:
        av.Packet: A properly formatted AV packet ready for muxing into the output stream
    """


    pkt = av.packet.Packet(bytearray(video_packet))
    pkt.stream = out_stream
    pkt.pts = pkt_data.pts
    pkt.dts = pkt_data.dts

    return pkt

In [ ]:
# Decoded Surface
surf_src = vali.Surface.Make(
    format=py_dec.Format,
    width=py_dec.Width,
    height=py_dec.Height,
    gpu_id=0)

# Resized Surface
surf_dst = vali.Surface.Make(
    format=py_dec.Format,
    width=target_w,
    height=target_h,
    gpu_id=0
)

# Numpy array which contains encoded packet
packet = np.ndarray(
    dtype=np.uint8,
    shape=())

pkt_data = vali.PacketData()

while True:
    # Decode single Surface
    success, details = py_dec.DecodeSingleSurface(surf_src, pkt_data)
    if not success:
        print(details)
        break

    # Resize
    success, details = py_res.Run(surf_src, surf_dst)
    if not success:
        print(details)
        break

    # Encode. If there's a compressed packet, write to file
    got_pkt = py_enc.EncodeSingleSurface(surf_dst, packet)
    if got_pkt:
        av_pkt = make_packet(packet, pkt_data, out_stream)
        dst_file.mux(av_pkt)

# Encoder is async, so we need to flush it in the end
while True:
    got_pkt = py_enc.FlushSinglePacket(packet)
    if got_pkt:
        av_pkt = make_packet(packet, pkt_data, out_stream)
        dst_file.mux(av_pkt)
    else:
        break

# Finish pending operations on file and close it
dst_file.close()